# Name: Fedi Ghanmi
## Overview : 
### This is a Data Exploration and Description presented with Visualisations graphs using Plotly Library.
### First, We will run our package import cell

In [ ]:
# installation of these packages is mandatory before importing them
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.io as pio 
pio.templates.default = "ggplot2"
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib inline
import json

### Our data consist of 3 product categories, with multiple sub-categories in it. They are sold to 3 customer segments all over USA. However, sales do not necessarily reflect how profitable a product is. We will proceed with a complete exploratory analysis to determine which is the most profitable product segment and in which state the superstore should invest its money in and maximize its profit.

In [ ]:
superstore = pd.read_csv("SampleSuperstore.csv")
superstore.head()

### For starter, we will examine our quantitative features correlation.

In [ ]:
correlation = superstore.corr()
fig = px.imshow(correlation, title ="Features Correlation Heatmap", color_continuous_scale=px.colors.sequential.BuGn)

fig.show()

### Sales and Profit for the store are linearly correlated with a pearson coefficient of ~ 0.6. The higher the sales, the higher profits alongside it. For the other features, it seems that the correlation does not give significant information. Most importantly, we do not have extreme multicollinearity between features in our dataset. We will explore other features to reveal this data secrets !

In [ ]:
category_profit = pd.pivot_table(superstore, values=["Profit"], index =["Category"] ,aggfunc="sum")
sorted_by_category_profit = pd.DataFrame(category_profit)
sorted_by_category_profit.reset_index(level = 0,inplace=True)

fig = go.Figure(data=[go.Pie(labels=sorted_by_category_profit["Category"]
                             , values=sorted_by_category_profit["Profit"], hole=.6)])
fig.update_traces(marker=dict(colors=px.colors.sequential.RdBu[:1]))
fig.update_layout(title="Donut Chart of Product Category Profit")


fig.show()

### Technology and Office Supplies are responsible for more than 90% of the store profit. Furniture Category represent the least profitable category. To have a more critical judgment, let us see in each category, which are the most profitable sub-categories.

In [ ]:
sub_category_profit = pd.pivot_table(superstore, values=["Profit"], index =["Category","Sub-Category"] ,aggfunc="sum")
sorted_by_sub_category_profit = pd.DataFrame(sub_category_profit)
sorted_by_sub_category_profit.reset_index(inplace=True)
sorted_by_sub_category_profit.sort_values(by="Profit", ascending=False,inplace = True)

fig = px.parallel_categories(sorted_by_sub_category_profit, dimensions=['Category', 'Sub-Category'],
                 color = "Profit",color_continuous_scale=px.colors.sequential.Blues, title ="Parallel Categories of Products")
fig.show()

### We can understand this graph by looking at the left side, which contains product categories. These categories are linked to their respective subcategories on the right, with the more blue the line, the higher the profit in that subcategory. The ones with the highest profit are the ones with the black color. On the right, the subcategories are sorted from highest to lowest profit realization.
### The 3 most profitable sub-categories belong in the Technology category. "Copiers, Phones and Acessoires" with about 50 thousand dollars each. Paper, Binders and Appliances product ranked second and belong to the Office Supplies, with an average of 40 thousand dollars each. Notice the products colored in white are incurring losses to the superstore. Tables, Bookcases and Supplies are not very successful. Our store will surely be better off without these products. But before we make any decision, let us examine why are such losses are taking places.
### If we look at the states in which these products are sold, we can extract extra insight that will guide us into our analysis and learn more about our store.

In [ ]:
# import GeoJSON File of US States in order to display a map of the US.
geo = json.load(open("../usa_states.json", "r"))

# This code creates a dataframe of profit, state and quantity sold sorted by the column State
table = pd.pivot_table(superstore, values=["Profit","Sales","Quantity"], index =["State"] ,aggfunc="sum")
sorted_by_state = pd.DataFrame(table)
sorted_by_state.reset_index(level = 0,inplace=True)

# Map display of profit per US State.
fig = px.choropleth_mapbox(sorted_by_state, geojson=geo, color="Profit",
                           locations="State", featureidkey="properties.NAME",
                           center={"lat": 37.0902, "lon": -95.7129},
                           mapbox_style="stamen-toner", zoom=3, opacity=0.7,
                           color_continuous_scale= px.colors.diverging.RdYlGn)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
print("Map Of Profit Realized per US States")
fig.show()

### If we hover over this Map, We notice California, New York and DC are representing the highest profits(colored in Green). Negative sum of profits in Texas, Ohio, Pennsylvania, and other states are colored in Red. Questions to be Asked here are: Why are these states making losses ? Is it related to the Product itself, to the State or maybe both are the cause ?
### One possible cause is the store's discount rate. To come to a conclusion, we will visualize which states endure the most discounts.

In [ ]:
# This code creates a dataframe of profit, state and quantity sold sorted by the column State
table_discount = pd.pivot_table(superstore, values=["Discount"], index =["State"] ,aggfunc="mean")
sorted_by_state_discount = pd.DataFrame(table_discount)
sorted_by_state_discount.reset_index(level = 0,inplace=True)

# Map display of profit per US State.
fig = px.choropleth_mapbox(sorted_by_state_discount, geojson=geo, color="Discount",
                           locations="State", featureidkey="properties.NAME",
                           center={"lat": 37.0902, "lon": -95.7129},
                           mapbox_style="stamen-toner", zoom=3, opacity=0.7,
                           color_continuous_scale='Reds')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0} )
print("Map Of Average Discount rate per US States")
fig.show()

### The average discount per state reveals that the shop offers substantial discounts in Texas, Ohio, Pennsylvania, and Illinois ( Same states that have negative profits ). As a result, we may deduce that the substantial losses are the result of the store's aggressive discounting in these states. However, the rationality behind the sales manager decision is uncertain. Why is he offering such steep discounts in these states? Is this because of the goods sold in these states? One probable hypothesis is that we are selling subpar products, thus management boosted discounts to stimulate sales. Nothing is known, as a result, we will divide our data into three categories. States with high profit yields, states with negative profits, and states with mid-profit yields. We will then assess our sales structure in relation to our segmentation and determine whether our discount is attributable to the product sold in these regions, the shipping mode, or even our client base.


In [ ]:
#select from our aggregated dataset the state that have average discount higher than 30% and subset our superstore dataframe.

neg_profit_states = sorted_by_state_discount.loc[
    sorted_by_state_discount["Discount"] >= 0.3,"State"]
neg_profit = superstore.loc[
    superstore["State"].isin(neg_profit_states.values.tolist()),]

#select from our aggregated dataset the state that have profits higher than 15k dollars and subset our superstore dataframe.

pos_profit_states = sorted_by_state.loc[
    sorted_by_state["Profit"] >= 15000,"State"]
pos_profit = superstore.loc[
    superstore["State"].isin(pos_profit_states.values.tolist()),]

#select from our aggregated dataset the state that have positive and medium profits and subset our superstore dataframe.

avg_profit_states = sorted_by_state.loc[
    (sorted_by_state["Profit"] < 9000) & (sorted_by_state["Profit"] > 0)  ,"State"]
avg_profit = superstore.loc[
    superstore["State"].isin(avg_profit_states.values.tolist()),]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x= pos_profit["Category"] , y = pos_profit["Sales"],
                           histfunc = "sum" , histnorm = "percent" , name="High Profit"))
fig.add_trace(go.Histogram(x= avg_profit["Category"] , y = avg_profit["Sales"],
                          histfunc = "sum" , histnorm = "percent" , name="Average Profit")) 
fig.add_trace(go.Histogram(x= neg_profit["Category"], y = neg_profit["Sales"],
                            histfunc = "sum" , histnorm = "percent", name="Negative Profit" )).update_xaxes(
    categoryorder='total descending')

fig.update_layout(bargap=0.5, bargroupgap=0.5, title="Histogram of Product Category Sturcture",
                 yaxis_title="% of Total Sales", xaxis_title="Product Category")

fig.show()

### Technology Products constitute around 35% of sales in High Profit States, 30% in Mid Profit States and 38% in Negative(losing) States. Same logic goes for Furniture and Office Supplies Products. We notice an average similarity in sales structure across yielding states ! That is what we will study for the consumer segment and shipment mode too.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x= pos_profit["Segment"] , y = pos_profit["Sales"],
                            histfunc = "sum", histnorm = "percent", name="High Profit"))
fig.add_trace(go.Histogram(x= avg_profit["Segment"] , y = avg_profit["Sales"],
                           histfunc = "sum", histnorm = "percent",name="Average Profit"))
fig.add_trace(go.Histogram(x= neg_profit["Segment"], y = neg_profit["Sales"],
                           histfunc = "sum", histnorm = "percent",name="Negative Profit")).update_xaxes(
    categoryorder='total descending')


fig.update_layout(bargap=0.5, bargroupgap=0.5, title="Histogram of Consumer Segments Sturcture",
                 yaxis_title="% of Total Sales", xaxis_title="Consumer Segment")

fig.show()

### In this histogram, for the Consumer Segment, we also notice similar sturcture for the 3 consumer segment across states.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x= pos_profit["Ship_Mode"] , y = pos_profit["Sales"],
                            histfunc = "sum", histnorm = "percent", name="High Profit"))
fig.add_trace(go.Histogram(x= avg_profit["Ship_Mode"] , y = avg_profit["Sales"],
                            histfunc = "sum", histnorm = "percent",name="Average Profit" ))
fig.add_trace(go.Histogram(x= neg_profit["Ship_Mode"], y = neg_profit["Sales"],
                           histfunc = "sum", histnorm = "percent",name="Negative Profit" )).update_xaxes(
    categoryorder='total descending')

fig.update_layout(bargap=0.5, bargroupgap=0.5, title="Histogram of Shipment Sturcture",
                 yaxis_title="% of Total Sales", xaxis_title="Shipment Mode")

fig.show()

### Again, these Histograms show that we have the same product structure, shipping structure and client structure on average across all three data splits. Our high discounts are neither attributable to the product sold in these regions nor the shipping mode or even our client base. So with a research I did, this may be a possible answer.
### According to https://www.tripstodiscover.com/americas-top-shopping-cities/ , Florida, Texas, and Illinois are well-known shopping locations. Large retail malls may potentially increase discounts by up to 50%. One possible reason behind our store high discounts is because competition for our supersotre may be fierce. We should recognise this fact and not to sell at steep discounts if the loss will outweight our profits. 
## Analysis Conclusion: 
### The store needs to take back its inventory in states causing negative profits since high discounts will only result in a detrimental influence on our enterprise. We should relocate all of our inventory in the before-mentionned states and sell them in states where competition is beatable by our store. these include California, Washingtnon and New York and Michigan. 
### Furthermore, the products casuing losses that we saw in the Parallel Categories of Products, such as Tables, Bookcases, and Fasteners, should be removed because their demand is low in every state. We should then use the extra resources to produce more of our successful categories, resulting in higher profits.
### Thank you for reading my analysis, if you have any questions feel free to contact me on linkedin !